In [ ]:
from huggingface_hub import snapshot_download
import os
hf_token = "====================================="
hf_repo_id = "IDEA-Research/grounding-dino-base"
project_name = "======"
BASE_ARTIFACT_URI = "gs://=======/hf"
os.system(f"rm -rf /tmp/model")
print("Start downloading")
snapshot_download(repo_id=hf_repo_id, token=hf_token, local_dir=f"/tmp/model")
print("Uploading")
os.system(f"gcloud storage cp /tmp/model/*.* {BASE_ARTIFACT_URI}/{hf_repo_id}")
print("Done")

In [ ]:
import os
REGION = "us-central1"
BASE_CONTAINER = f"{REGION}-docker.pkg.dev/{project_name}/base/transformer:v1"
PACKAGE_PATH = "package"
dockerfile = """
FROM nvidia/cuda:12.4.1-runtime-ubuntu22.04
RUN apt-get -y update
RUN apt-get -y install git
RUN apt-get -y install python3 python3-pip
RUN ln -s /usr/bin/python3 /usr/bin/python
RUN pip install torch==2.2.2
RUN pip install accelerate==0.29.2
RUN pip install transformers==4.39.3
ENV VERTEX_CPR_MAX_WORKERS 1
"""
with open(f"Dockerfile", "w") as image_file:
    image_file.write(dockerfile)
image_file.close()

base_container_build = f'''
gcloud builds submit --region={REGION} --tag={BASE_CONTAINER} --machine-type=E2_HIGHCPU_32
'''
os.system(base_container_build)

In [ ]:
!gcloud auth configure-docker us-central1-docker.pkg.dev --quiet

In [ ]:
image = Image.open("car.png").convert("RGB")
image.size[::-1]

In [ ]:
#setting below turns all container build and localmodel logs
#import logging
#logging.basicConfig(level=logging.INFO)
from google.cloud.aiplatform.prediction import LocalModel
#add custom predictor
vai_serving_container_uri = f"{REGION}-docker.pkg.dev/{project_name}/custom-inference-gpu/vai-transformer-grounding-dino"
#must secure sufficient space
from package.VAIGroundingDINO import VAIGroundingDINO
local_model = LocalModel.build_cpr_model(
    src_dir=PACKAGE_PATH,
    output_image_uri=vai_serving_container_uri,
    predictor=VAIGroundingDINO,
    requirements_path=f"{PACKAGE_PATH}/requirements.txt",
    #extra_packages=["deploy_package/custom_package.tar.gz"]
    base_image=f"--platform=linux/amd64 {BASE_CONTAINER} AS build",
    no_cache = False
)

In [ ]:
import json
import logging
logging.basicConfig(level=logging.INFO)
test_artifact_uri = f"{BASE_ARTIFACT_URI}/{hf_repo_id}"
with local_model.deploy_to_local_endpoint(
        artifact_uri=test_artifact_uri,
        gpu_count=-1
    ) as local_endpoint:
    health_check_response = local_endpoint.run_health_check()
    request = {
        "instances" : [
            {
            "image_uri": "car.png",
            "prompt": "a car",
            "box_threshold": 0.4,
            "text_threshold": 0.3
            }
        ]
    }
    for i in range(1):
        predict_response = local_endpoint.predict(
            request=json.dumps(request),
            headers={"Content-Type": "application/json"},
        )
    local_endpoint.print_container_logs(show_all=True)
    print(health_check_response, health_check_response.content)
    print(predict_response)

In [ ]:
masks = predict_response.json()['predictions'][0]['masks']
width = 640
height = 480
width = 2646
height = 1764

In [ ]:
predict_response.json()

In [ ]:
import matplotlib.pyplot as plt
#image = torch.zeros(480, 640, 3, dtype=torch.uint8) #black color
#image[:, :, :] = 255 #white color
import numpy as np
base = np.zeros((height, width, 3), np.uint8) #black color
base[:] = (255, 255, 255) #white color
for mask in masks:
    #image = np.array(mask).reshape(height, width, 1)
    #np.copyto(base, image, where=(image != 0))
    base += np.array(mask).reshape(height, width, 1)
base = 255 - base #inverse color
plt.imshow(base)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np    

output = np.array(raw_image) / 255
for mask in masks:
    color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    image = (np.array(mask).reshape(height, width, 1) * color.reshape(1, 1, -1))[:, :, :3]
    np.copyto(output, image, where=(image != 0))
plt.imshow(output)

In [ ]:
from PIL import Image
Image.fromarray((output * 255).astype(np.uint8)).save('output.png')

In [ ]:
local_model.push_image()

In [ ]:
from google.cloud import aiplatform
aiplatform.Model.upload(
    display_name = "ground_dino_bbox",
    local_model = local_model,
    artifact_uri = test_artifact_uri,
    #parent_model = prev_model.resource_name,
    #is_default_version=True,
    serving_container_environment_variables={
        # Optional env var so that `uvicorn` only runs the model in 1 worker
        "VERTEX_CPR_MAX_WORKERS": 1,
    },
)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

plt.imshow(np.array(image))
ax = plt.gca()
for mask in masks[0]:
    show_mask(mask, ax=ax, random_color=True)
plt.axis("off")
plt.show()

In [ ]:
from PIL import Image
import numpy as np

def get_mask(mask):
    color = np.array([0, 0, 0, 0.9])
    h, w = mask.shape[-2:]
    return mask.reshape(h, w, 1) * color.reshape(1, 1, -1)

generated_masks = []
for mask in masks[0]:
    generated_masks.append(get_mask(mask))